In [ ]:
import ijson

def extract_questions(file_path):
    questions = []
    with open(file_path, 'r', encoding='utf-8') as file:
        parser = ijson.parse(file)
        for prefix, event, value in parser:
            if prefix.endswith('.question') and event == 'string':
                questions.append(value)
    return questions

qa_file_path = '/home/eternal/qa_train.json'
questions = extract_questions(qa_file_path)
print(questions[:10])  # 추출된 질문 일부 출력



In [ ]:
def extract_relevant_sentences(questions, wiki_file_path, output_file_path):
    relevant_sentences = []
    with open(wiki_file_path, 'r', encoding='utf-8') as wiki_file:
        for line in wiki_file:
            for question in questions:
                if any(word in line for word in question.split()):
                    relevant_sentences.append(line)
                    break

    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        for sentence in relevant_sentences:
            output_file.write(sentence)

    return relevant_sentences

wiki_file_path = '/home/eternal/processed_wikipedia.txt/processed_wikipedia.txt'
output_file_path='/home/eternal/processed_wikipedia.txt/relevant_sentences.txt'
relevant_sentences = extract_relevant_sentences(questions, wiki_file_path, output_file_path)
print(f"Extracted {len(relevant_sentences)} relevant sentences.")

위의 코드는 Q와 맞는 것만 txt에서 추출까지만 합니다. -> 이게 그 텍스트의 정체입니다.
JSON 파일을 스트리밍 방식으로 파싱하기 위해 사용됩니다. 이는 특히 대용량 JSON 파일을 처리할 때 유용합니다.
extract_questions 함수:

JSON 파일(file_path)에서 'question' 키를 가진 모든 문자열 값을 추출하여 리스트로 반환합니다.
ijson.parse(file)를 사용하여 JSON 파일을 파싱하고, 'question' 키가 발견될 때마다 그 값을 리스트에 추가합니다.
extract_relevant_sentences 함수:

위에서 추출한 질문 리스트(questions)와 위키피디아 텍스트 파일(wiki_file_path)을 입력으로 받아, 질문과 관련된 문장을 찾습니다.
위키피디아 텍스트 파일을 한 줄씩 읽어, 각 줄에 질문의 단어들이 포함되어 있는지 검사합니다. 포함된 문장은 relevant_sentences 리스트에 추가됩니다.
관련된 문장들을 output_file_path에 저장하고, 관련 문장 리스트를 반환합니다.

In [6]:
import ijson
import re
from difflib import SequenceMatcher
import nltk
from nltk.corpus import words
from spellchecker import SpellChecker
import sys
import time
from multiprocessing import Pool, cpu_count

# Ensure nltk word list is downloaded
nltk.download('words')

[nltk_data] Downloading package words to /home/eternal/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [7]:
#질문 추출

def extract_questions(file_path):
    questions = []
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            parser = ijson.parse(file)
            for prefix, event, value in parser:
                if prefix.endswith('.question') and event == 'string':
                    questions.append(value)
    except FileNotFoundError:
        print("Error: File not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
    return questions


ijson 라이브러리를 사용하여 JSON 파일을 스트리밍 방식으로 파싱실시 //
.question 키를 가진 이벤트를 찾으면 해당 값을 questions 리스트에 추가 //
예외 처리를 통해 파일이 없거나 다른 문제가 발생할 경우 오류 메시지를 출력

In [8]:
#텍스트 전처리 및 맞춤법 교정 (preprocess_text, correct_spelling 함수)
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)
    return text
def correct_spelling(text):
    spell = SpellChecker()
    corrected_text = [spell.correction(word) if spell.correction(word) else word for word in text.split()]
    return ' '.join(corrected_text)


preprocess_text 함수는 텍스트를 소문자로 변환하고 불필요한 문자를 제거
(모든 문자를 소문자로 변환,
정규 표현식을 사용하여 단어 이외의 모든 문자를 공백으로 대체).//


correct_spelling 함수는 SpellChecker 라이브러리를 사용하여 텍스트의 맞춤법을 교정(SpellChecker 객체를 생성.
텍스트의 각 단어를 순회하며 맞춤법을 교정합니다. 교정된 단어가 없을 경우 원래 단어를 사용,
교정된 단어들을 공백으로 연결하여 반환)




In [9]:
#관련 문장 판별 (is_relevant_sentence 함수)
from difflib import SequenceMatcher

def is_relevant_sentence(question, sentence, threshold=0.6):
    question = correct_spelling(preprocess_text(question))
    sentence = correct_spelling(preprocess_text(sentence))
    similarity = SequenceMatcher(None, question, sentence).ratio()
    return similarity > threshold


preprocess_text와 correct_spelling 함수를 사용하여 질문과 문장을 전처리하고 맞춤법을 교정//.
SequenceMatcher를 사용하여 질문과 문장의 유사도를 계산//
유사도가 임계값(threshold)보다 높으면 True를 반환하여 관련 문장으로 판별

In [10]:
#청크 단위로 문장 처리 (process_chunk 함수)
def process_chunk(questions, chunk, threshold):
    processed_chunk = []
    for line in chunk:
        processed_line = line.strip()
        for question in questions:
            if is_relevant_sentence(question, processed_line, threshold):
                processed_line = correct_spelling(preprocess_text(processed_line))
                break
        processed_chunk.append(processed_line)
    return processed_chunk


입력으로 받은 청크 내의 각 문장을 순회//
각 문장에 대해 질문과의 유사도를 계산하여 관련 문장으로 판별되면, 해당 문장을 맞춤법 교정 및 전처리//
모든 문장을 처리한 후, 수정된 문장 리스트를 반환

In [11]:
#관련 문장 추출 및 원본 파일 수정 (extract_and_replace_relevant_sentences 함수)
def extract_and_replace_relevant_sentences(questions, wiki_file_path, output_file_path, chunk_size=100000, num_workers=cpu_count()):
    total_lines = sum(1 for _ in open(wiki_file_path, 'r', encoding='utf-8'))

    start_time = time.time()
    with open(wiki_file_path, 'r', encoding='utf-8') as wiki_file, open(output_file_path, 'w', encoding='utf-8') as output_file, Pool(num_workers) as pool:
        chunk = []
        for i, line in enumerate(wiki_file):
            chunk.append(line)
            if (i + 1) % chunk_size == 0 or i + 1 == total_lines:
                try:
                    elapsed_time = time.time() - start_time
                    lines_processed = i + 1
                    remaining_lines = total_lines - lines_processed
                    estimated_total_time = (elapsed_time / lines_processed) * total_lines
                    estimated_remaining_time = estimated_total_time - elapsed_time
                    print(f"Processing line {lines_processed}/{total_lines}")
                    print(f"Elapsed time: {elapsed_time:.2f} seconds")
                    print(f"Estimated remaining time: {estimated_remaining_time / 3600:.2f} hours")
                    sys.stdout.flush()

                    results = pool.starmap(process_chunk, [(questions, chunk, 0.6)])
                    for result in results:
                        for processed_line in result:
                            output_file.write(processed_line + '\n')
                    chunk = []
                except Exception as e:
                    print(f"Error processing chunk at line {i+1}: {e}")
                    sys.stdout.flush()

        # Process remaining lines if any
        if chunk:
            try:
                results = pool.starmap(process_chunk, [(questions, chunk, 0.6)])
                for result in results:
                    for processed_line in result:
                        output_file.write(processed_line + '\n')
            except Exception as e:
                print(f"Error processing final chunk: {e}")
                sys.stdout.flush()

    elapsed_time = time.time() - start_time
    print(f"Total elapsed time: {elapsed_time:.2f} seconds")


파일의 총 라인 수를 계산//
원본 파일을 읽고, 수정된 내용을 새로운 파일에 사용//
병렬 처리를 위해 Pool 객체를 생성하고, 청크 단위로 문장을 처리//
각 청크가 처리될 때마다 진행 상황을 출력//
모든 청크가 처리된 후, 새 파일을 원본 파일로 대체

In [12]:
qa_file_path = '/home/eternal/qa_train.json'
wiki_file_path = '/home/eternal/processed_wikipedia.txt/processed_wikipedia.txt'
output_file_path = '/home/eternal/processed_wikipedia.txt/processed_wikipedia_filtered.txt'

# 질문 추출
print("Extracting questions...")
questions = extract_questions(qa_file_path)
print(f"Extracted {len(questions)} questions.")
print(questions[:10])  # 추출된 질문 일부 출력

# 관련 문장 추출 및 저장
print("Extracting relevant sentences...")
extract_and_replace_relevant_sentences(questions, wiki_file_path, output_file_path)

# 기존 파일 삭제하고 새 파일로 대체
import os
print("Replacing original file with the filtered file...")
os.remove(wiki_file_path)
os.rename(output_file_path, wiki_file_path)
print("Replacement complete.")

Extracting questions...
Extracted 88066 questions.
['In what year the the venue that Marcia White is president of open?', 'What country is home to the sports club loaning Bruno Paulista to Vasco da Gama?', '"Southern Air" featured Ray Stevens, Minnie Pearl and what other Southern comedian?', 'The Treaty of Prairie du Chien may refer to any of several treaties made between the United States and representatives from the native american people who called themselves what?', 'What profession do Thomas Merton and Michael Moorcock have in common?', 'Who was the first person to describe Relational Quantum mechanics and has worked in Italy, the United States, and France?', 'Where is the Silver Dollar city theme park with the Grand exposition  steel roller coaster located?', 'Finna Get Loose is a song by Puff Daddy and what American rapper who is also a record and film producer?', 'What town is Hore Abbey located near?', 'In what year was the actress who was starred in "The Telling" with Holly M

## 여기서 부터 입니다.##

In [2]:
import ijson
import re
from difflib import SequenceMatcher
import nltk
from spellchecker import SpellChecker
import sys
import time
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

# Ensure nltk word list is downloaded
nltk.download('words')

def extract_questions(file_path):
    questions = []
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            parser = ijson.parse(file)
            for prefix, event, value in parser:
                if prefix.endswith('.question') and event == 'string':
                    questions.append(value)
    except FileNotFoundError:
        print("Error: File not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
    return questions

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)
    return text

def correct_spelling(text):
    spell = SpellChecker()
    corrected_text = [spell.correction(word) if spell.correction(word) else word for word in text.split()]
    return ' '.join(corrected_text)

def is_relevant_sentence(question, sentence, threshold=0.6):
    question = preprocess_text(question)
    sentence = preprocess_text(sentence)
    similarity = SequenceMatcher(None, question, sentence).ratio()
    return similarity > threshold

def process_chunk(args):
    questions, chunk, threshold = args
    processed_chunk = []
    for line in chunk:
        processed_line = line.strip()
        for question in questions:
            if is_relevant_sentence(question, processed_line, threshold):
                processed_line = correct_spelling(processed_line)
                break
        processed_chunk.append(processed_line)
    return processed_chunk

def extract_and_replace_relevant_sentences(questions, wiki_file_path, output_file_path, chunk_size=100, num_workers=max(1, cpu_count() // 4)):
    total_lines = sum(1 for _ in open(wiki_file_path, 'r', encoding='utf-8'))
    print(f"Total lines in file: {total_lines}")

    start_time = time.time()
    chunk = []
    results = []

    with open(wiki_file_path, 'r', encoding='utf-8') as wiki_file:
        for i, line in enumerate(wiki_file):
            chunk.append(line)
            if (i + 1) % chunk_size == 0 or i + 1 == total_lines:
                results.append(chunk)
                chunk = []

    print(f"Total chunks to process: {len(results)}")

    processed_chunks = []
    with Pool(num_workers) as pool:
        for result in tqdm(pool.imap_unordered(process_chunk, [(questions, chunk, 0.6) for chunk in results]), total=len(results)):
            processed_chunks.append(result)

    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        for processed_chunk in tqdm(processed_chunks, desc="Writing to file"):
            for processed_line in processed_chunk:
                output_file.write(processed_line + '\n')

    elapsed_time = time.time() - start_time
    print(f"Total elapsed time: {elapsed_time:.2f} seconds")

# 파일 경로 설정
qa_file_path = '/home/eternal/this/qa_train.json'
wiki_file_path = '/home/eternal/this/processed_wikipedia.txt/processed_wikipedia.txt'
output_file_path = '/home/eternal/this/processed_wikipedia_filtered.txt'

# 질문 추출
print("Extracting questions...")
questions = extract_questions(qa_file_path)
print(f"Extracted {len(questions)} questions.")
print(questions[:10])  # 추출된 질문 일부 출력

# 관련 문장 추출 및 저장
print("Extracting relevant sentences...")
extract_and_replace_relevant_sentences(questions, wiki_file_path, output_file_path)



'''
# 기존 파일 삭제하고 새 파일로 대체
import os
print("Replacing original file with the filtered file...")
os.remove(wiki_file_path)
os.rename(output_file_path, wiki_file_path)
print("Replacement complete.")
'''

[nltk_data] Downloading package words to /home/eternal/nltk_data...
[nltk_data]   Package words is already up-to-date!


Extracting questions...
Extracted 88066 questions.
['In what year the the venue that Marcia White is president of open?', 'What country is home to the sports club loaning Bruno Paulista to Vasco da Gama?', '"Southern Air" featured Ray Stevens, Minnie Pearl and what other Southern comedian?', 'The Treaty of Prairie du Chien may refer to any of several treaties made between the United States and representatives from the native american people who called themselves what?', 'What profession do Thomas Merton and Michael Moorcock have in common?', 'Who was the first person to describe Relational Quantum mechanics and has worked in Italy, the United States, and France?', 'Where is the Silver Dollar city theme park with the Grand exposition  steel roller coaster located?', 'Finna Get Loose is a song by Puff Daddy and what American rapper who is also a record and film producer?', 'What town is Hore Abbey located near?', 'In what year was the actress who was starred in "The Telling" with Holly M

  0%|                                               | 0/1055025 [00:32<?, ?it/s]Process ForkPoolWorker-12:
Process ForkPoolWorker-14:
Process ForkPoolWorker-11:
Process ForkPoolWorker-16:
Process ForkPoolWorker-13:
Process ForkPoolWorker-9:
Process ForkPoolWorker-10:
Process ForkPoolWorker-15:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _boots

KeyboardInterrupt: 

In [ ]:
import json
import re
import torch
from sentence_transformers import SentenceTransformer, util
from spellchecker import SpellChecker
import language_tool_python
from tqdm import tqdm

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# SentenceTransformer 모델 로드
model = SentenceTransformer('all-MiniLM-L6-v2').to(device)

# 언어 도구와 맞춤법 검사기 설정
spell = SpellChecker()
tool = language_tool_python.LanguageTool('en-US')

# 텍스트 정제 함수
def clean_text(text):
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)  # 여러 개의 공백을 단일 공백으로 대체
    text = re.sub(r'[^\w\s.,!?]', '', text)  # 특수 문자 제거
    return text

# 오탈자 수정 함수
def correct_spelling(text):
    corrected_text = []
    for word in text.split():
        corrected_word = spell.correction(word)
        if corrected_word is None:
            corrected_text.append(word)
        else:
            corrected_text.append(corrected_word)
    return ' '.join(corrected_text)

# 문법 검사 및 수정 함수
def correct_grammar(text):
    matches = tool.check(text)
    corrected_text = language_tool_python.utils.correct(text, matches)
    return corrected_text

# 전체 텍스트 정제 함수
def preprocess_text(text):
    text = clean_text(text)
    text = correct_spelling(text)
    text = correct_grammar(text)
    return text

# 문장 임베딩 생성 함수
def embed_texts(texts):
    return model.encode(texts, convert_to_tensor=True, device=device)

# 유사도 계산 함수
def calculate_similarity(question_embedding, document_embeddings):
    return util.pytorch_cos_sim(question_embedding, document_embeddings)

# 데이터 로드 및 유사도 기반 전처리
def load_and_clean_data(file_path, top_k=5):
    with open(file_path, "r") as file:
        data = json.load(file)
    
    clean_data = []
    for item in tqdm(data, desc="Processing data"):
        question = clean_text(item["question"])
        question_embedding = embed_texts([question])
        
        answers = [clean_text(answer) for answer in item.get("answers", [])]
        if not answers:
            continue
        
        answer_embeddings = embed_texts(answers)
        similarities = calculate_similarity(question_embedding, answer_embeddings)
        
        top_indices = similarities.argsort(descending=True).tolist()[0][:top_k]
        for idx in top_indices:
            clean_item = {
                "question": question,
                "answer": preprocess_text(answers[idx])
            }
            clean_data.append(clean_item)
    
    return clean_data

train_data = load_and_clean_data("/home/eternal/this/qa_train.json")
test_data = load_and_clean_data("/home/eternal/this/qa_test.json")

with open("/home/eternal/this/processed_wikipedia.txt/processed_wikipedia.txt", "r") as file:
    knowledge_base = [preprocess_text(line) for line in tqdm(file, desc="Processing knowledge base")]


Using device: cuda



Processing data:  44%|████████▎          | 38473/88066 [01:09<01:27, 567.35it/s]

In [ ]:
import json
import re
import torch
from sentence_transformers import SentenceTransformer, util
from spellchecker import SpellChecker
import language_tool_python
from tqdm import tqdm

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# SentenceTransformer 모델 로드
model = SentenceTransformer('all-MiniLM-L6-v2').to(device)

# 언어 도구와 맞춤법 검사기 설정
spell = SpellChecker()
tool = language_tool_python.LanguageTool('en-US')

# 텍스트 정제 함수
def clean_text(text):
    text = text.strip()  # 텍스트의 양쪽 끝에 있는 불필요한 공백 제거
    text = re.sub(r'\s+', ' ', text)  # 텍스트 내부의 여러 개의 공백을 단일 공백으로 변환
    text = re.sub(r'[^\w\s.,!?]', '', text)  # 알파벳, 숫자, 공백, 쉼표, 마침표, 느낌표, 물음표를 제외한 모든 특수 문자 제거
    return text

# 오탈자 수정 함수
def correct_spelling(text):
    corrected_text = []
    for word in text.split():
        corrected_word = spell.correction(word)  # 각 단어의 맞춤법을 검사하여 올바른 단어로 교정
        if corrected_word is None:
            corrected_text.append(word)
        else:
            corrected_text.append(corrected_word)
    return ' '.join(corrected_text)

# 문법 검사 및 수정 함수
def correct_grammar(text):
    matches = tool.check(text)  # 텍스트를 검사하여 문법 오류를 찾음
    corrected_text = language_tool_python.utils.correct(text, matches)  # 발견된 문법 오류를 수정하여 올바른 문장으로 변환
    return corrected_text

# 전체 텍스트 정제 함수
def preprocess_text(text):
    text = clean_text(text)  # 공백 및 특수 문자 제거
    text = correct_spelling(text)  # 맞춤법 교정
    text = correct_grammar(text)  # 문법 검사 및 수정
    return text

# 문장 임베딩 생성 함수
def embed_texts(texts):
    return model.encode(texts, convert_to_tensor=True, device=device)

# 유사도 계산 함수
def calculate_similarity(question_embedding, document_embeddings):
    return util.pytorch_cos_sim(question_embedding, document_embeddings)

# 데이터 로드 및 유사도 기반 전처리
def load_and_clean_data(file_path, top_k=5):
    with open(file_path, "r") as file:
        data = json.load(file)
    
    clean_data = []
    for item in tqdm(data, desc="Processing data"):
        question = clean_text(item["question"])  # 질문 텍스트 정제
        question_embedding = embed_texts([question])  # 질문 문장 임베딩 생성
        
        answers = [clean_text(answer) for answer in item.get("answers", [])]  # 답변 텍스트 정제
        if not answers:
            continue
        
        answer_embeddings = embed_texts(answers)  # 답변 문장 임베딩 생성
        similarities = calculate_similarity(question_embedding, answer_embeddings)  # 질문과 답변 간의 유사도 계산
        
        top_indices = similarities.argsort(descending=True).tolist()[0][:top_k]  # 유사도가 높은 상위 답변 선택
        for idx in top_indices:
            clean_item = {
                "question": question,
                "answer": preprocess_text(answers[idx])  # 답변 텍스트 정제
            }
            clean_data.append(clean_item)
    
    return clean_data

# 트레인 데이터와 테스트 데이터 로드 및 전처리
train_data = load_and_clean_data("/home/eternal/this/qa_train.json")
test_data = load_and_clean_data("/home/eternal/this/qa_test.json")

# 지식 기반 문서 로드 및 전처리
with open("/home/eternal/this/processed_wikipedia.txt/processed_wikipedia.txt", "r") as file:
    knowledge_base = [preprocess_text(line) for line in tqdm(file, desc="Processing knowledge base")]  # 지식 기반 문서 정제


Using device: cuda




Processing data:   0%|                                | 0/88066 [00:00<?, ?it/s]

Processing data:   0%|                      | 54/88066 [00:00<02:43, 538.55it/s]

Processing data:   0%|                     | 109/88066 [00:00<02:41, 543.65it/s]

Processing data:   0%|                     | 165/88066 [00:00<02:39, 549.72it/s]

Processing data:   0%|                     | 221/88066 [00:00<02:39, 552.39it/s]

Processing data:   0%|                     | 278/88066 [00:00<02:37, 558.28it/s]

Processing data:   0%|                     | 335/88066 [00:00<02:36, 560.24it/s]

Processing data:   0%|                     | 392/88066 [00:00<02:35, 562.06it/s]

Processing data:   1%|                     | 449/88066 [00:00<02:36, 561.54it/s]

Processing data:   1%|                     | 506/88066 [00:00<02:37, 557.52it/s]

Processing data:   1%|▏                    | 562/88066 [00:01<02:36, 557.55it/s]

Processing data:   1%|▏                    | 618/88066 [00:01<02:37, 553.72it/s]

Processing dat

# 설명을 포함한 텍스트 파일 생성
explanation = """
### 각 함수의 노이즈 처리 설명

1. **clean_text 함수**

- **양쪽 끝 공백 제거**:
    ```python
    text = text.strip()
    ```
    - 텍스트의 양쪽 끝에 있는 불필요한 공백을 제거합니다.

- **여러 개의 공백을 단일 공백으로 변환**:
    ```python
    text = re.sub(r'\s+', ' ', text)
    ```
    - 텍스트 내부의 여러 개의 공백(탭, 줄바꿈 포함)을 단일 공백으로 변환합니다.

- **특수 문자 제거**:
    ```python
    text = re.sub(r'[^\w\s.,!?]', '', text)
    ```
    - 알파벳, 숫자, 공백, 쉼표, 마침표, 느낌표, 물음표를 제외한 모든 특수 문자를 제거합니다.

2. **correct_spelling 함수**

- **맞춤법 교정**:
    ```python
    corrected_word = spell.correction(word)
    ```
    - 각 단어의 맞춤법을 검사하여 올바른 단어로 교정합니다. 교정된 단어가 없을 경우 원래 단어를 사용합니다.

- **단어 조합**:
    ```python
    corrected_text.append(corrected_word)
    ```
    - 맞춤법이 교정된 단어들을 조합하여 최종 텍스트를 생성합니다.

3. **correct_grammar 함수**

- **문법 검사**:
    ```python
    matches = tool.check(text)
    ```
    - 텍스트를 검사하여 문법 오류를 찾습니다.

- **문법 오류 수정**:
    ```python
    corrected_text = language_tool_python.utils.correct(text, matches)
    ```
    - 발견된 문법 오류를 수정하여 올바른 문장으로 변환합니다.

4. **preprocess_text 함수**

- **공백 및 특수 문자 제거** (`clean_text`):
    ```python
    text = clean_text(text)
    ```

- **맞춤법 교정** (`correct_spelling`):
    ```python
    text = correct_spelling(text)
    ```

- **문법 검사 및 수정** (`correct_grammar`):
    ```python
    text = correct_grammar(text)
    ```

이 코드는 질문과 답변 데이터를 정제하여 기존 데이터를 대체하며, 문장 임베딩과 유사도 계산을 통해 가장 관련성 높은 답변을 선택합니다. 또한, 지식 기반 문서에서 각 문장을 정제하여 데이터 품질을 높입니다.
"""

# 텍스트 파일로 저장
with open("/home/eternal/this/explanation.txt", "w") as file:
    file.write(explanation)
